## Release of hammer-cli gem

### Requirements
- push access to https://github.com/theforeman/hammer-cli
- push access to rubygems.org for hammer-cli
- sudo yum install transifex-client python-slugify asciidoc
- ensure neither the `git push` or `gem push` don't require interractive auth. If you can't use api key or ssh key to auth skip these steps and run them form the shell manually 
- to push translations you need an account on Transifex (https://docs.transifex.com/client/client-configuration#~/-transifexrc)

### Release process
- Follow the steps with `<Shift>+<Enter>` or `<Ctrl>+<Enter>,<Down>`
- If anything fails, fix it and re-run the step if applicable

### Release settings

In [ ]:
%cd ..

### Update the following notebook settings

In [1]:
NEW_VERSION = '3.18.0'
LAST_VERSION = '3.17.0'
DEVELOP_VERSION = '3.19.0-develop'
NEXT_FUTURE_VERSION = '3.19.0'
MAJOR_RELEASE = True
STABLE_BRANCH = '3.18-stable'
GIT_REMOTE_UPSTREAM = 'origin'
WORK_BRANCH = 'master' if MAJOR_RELEASE else STABLE_BRANCH
CHERRY_PICKS = []


### Ensure the repo is up to date

In [2]:
! git checkout {WORK_BRANCH}

Already on 'master'
Your branch is up to date with 'origin/master'.


In [3]:
! git fetch {GIT_REMOTE_UPSTREAM}

In [4]:
! git rebase {GIT_REMOTE_UPSTREAM}/{WORK_BRANCH}

Current branch master is up to date.


### Cherry picks for minor release

In [5]:
if not MAJOR_RELEASE:
    for cp in CHERRY_PICKS:
        ! git cherry-pick -x {cp}

### Run tests localy

In [6]:
! bundle update

Fetching gem metadata from https://rubygems.org/...........
Resolving dependencies...
/usr/share/gems/gems/rdoc-6.14.0/lib/rdoc/version.rb:8: warning: already initialized constant RDoc::VERSION
/home/evgeni/.gem/ruby/gems/rdoc-7.1.0/lib/rdoc/version.rb:8: warning: previous definition of VERSION was here
/usr/share/gems/gems/rdoc-6.14.0/lib/rdoc.rb:68: warning: already initialized constant RDoc::VISIBILITIES
/home/evgeni/.gem/ruby/gems/rdoc-7.1.0/lib/rdoc.rb:68: warning: previous definition of VISIBILITIES was here
/usr/share/gems/gems/rdoc-6.14.0/lib/rdoc.rb:74: warning: already initialized constant RDoc::DOT_DOC_FILENAME
/home/evgeni/.gem/ruby/gems/rdoc-7.1.0/lib/rdoc.rb:74: warning: previous definition of DOT_DOC_FILENAME was here
/usr/share/gems/gems/rdoc-6.14.0/lib/rdoc.rb:79: warning: already initialized constant RDoc::GENERAL_MODIFIERS
/home/evgeni/.gem/ruby/gems/rdoc-7.1.0/lib/rdoc.rb:79: warning: previous definition of GENERAL_MODIFIERS was here
/usr/share/gems/gems/rdoc-6.14.0

In [7]:
! bundle exec rake test

(in /home/evgeni/Devel/theforeman/hammer-cli)
/usr/bin/ruby -I"lib:lib" /home/evgeni/.gem/ruby/gems/rake-13.3.1/lib/rake/rake_test_loader.rb "test/functional/defaults_test.rb" "test/functional/help_test.rb" "test/functional/nil_values_test.rb" "test/unit/abstract_test.rb" "test/unit/apipie/api_connection_test.rb" "test/unit/apipie/command_test.rb" "test/unit/apipie/option_builder_test.rb" "test/unit/apipie/option_definition_test.rb" "test/unit/bash_test.rb" "test/unit/ca_cert_manager_test.rb" "test/unit/command_extensions_test.rb" "test/unit/completer_test.rb" "test/unit/connection_test.rb" "test/unit/csv_parser_test.rb" "test/unit/defaults_test.rb" "test/unit/exception_handler_test.rb" "test/unit/help/builder_test.rb" "test/unit/help/definition/abstract_item_test.rb" "test/unit/help/definition/list_test.rb" "test/unit/help/definition/note_test.rb" "test/unit/help/definition/section_test.rb" "test/unit/help/definition/text_test.rb" "test/unit/help/definition_test.rb" "test/unit/help/te

### Update release related stuff

In [14]:
! pwd

/home/evgeni/Devel/theforeman/hammer-cli/rel-eng


In [15]:
! cd .. && sed -i 's/Gem::Version.new .*/Gem::Version.new "{NEW_VERSION}"/' lib/hammer_cli/version.rb

In [16]:
# Parse git changelog
from IPython.display import Markdown as md
from subprocess import check_output
from shlex import split
import re

def format_log_entry(entry):
    issues = re.findall(r'[^(]#([0-9]+)', entry)
    entry = re.sub(r'([fF]ixes|[rR]efs)[^-]*-\s*(.*)', r'\2', entry)
    entry = '* ' + entry.capitalize()
    entry = re.sub(r'\(#([0-9]+)\)', r'([PR #\1](https://github.com/theforeman/hammer-cli/pull/\1))', entry)
    for i in issues:
        referenced_issues.append(i)
        entry = entry + ', [#%s](http://projects.theforeman.org/issues/%s)' % (i, i)
    return entry

def skip(entry):
    if re.match(r'Merge pull', entry) or \
      re.match(r'^i18n', entry) or \
      re.match(r'^Bump to version', entry):
        return True
    else:
        return False
referenced_issues = []    
git_log_cmd = 'git log --pretty=format:"%%s" %s..HEAD' % LAST_VERSION
log = check_output(split(git_log_cmd)).decode('utf8').split('\n')
change_log = [format_log_entry(e) for e in log if not skip(e)]
md('\n'.join(change_log))


* Bump to 3.18.0-develop

In [ ]:
# Write release notes
from datetime import datetime
import fileinput
import sys

fh = fileinput.input('doc/release_notes.md', inplace=True)  
for line in fh:  
    print(line.rstrip())
    if re.match(r'========', line):
        print('### %s (%s)' % (NEW_VERSION, datetime.today().strftime('%Y-%m-%d')))
        for entry in change_log:
            print(entry)
        print('')
fh.close()  

#### Manual step: Update deps in the gemspec if neccessary

### Check what is going to be commited

In [ ]:
! git add -u
! git status

In [ ]:
! git diff --cached

### Commit changes

In [ ]:
! git commit -m "Bump to {NEW_VERSION}"

### Update translations

In [ ]:
if MAJOR_RELEASE:
    ! bundle exec rake tx:update

### Tag new version

In [ ]:
! git tag {NEW_VERSION}

### Prepare stable branch for major release

In [ ]:
if MAJOR_RELEASE:
    ! git checkout -b {STABLE_BRANCH}
    ! git push {GIT_REMOTE_UPSTREAM} {STABLE_BRANCH}
    ! git checkout {WORK_BRANCH}

### Build the gem

In [ ]:
! rake build

In [ ]:
! gem push pkg/hammer_cli-{NEW_VERSION}.gem

### Bump the develop version for major release

In [ ]:
if MAJOR_RELEASE:
    ! sed -i 's/Gem::Version.new .*/Gem::Version.new "{DEVELOP_VERSION}"/' lib/hammer_cli/version.rb

In [ ]:
if MAJOR_RELEASE:
    ! git add -u
    ! git status

In [ ]:
if MAJOR_RELEASE:
    ! git diff --cached

In [ ]:
if MAJOR_RELEASE:
    ! git commit -m "Bump to {DEVELOP_VERSION}"

### PUSH the changes upstream If everything is correct

In [ ]:
! git push {GIT_REMOTE_UPSTREAM} {WORK_BRANCH}

In [ ]:
! git push --tags {GIT_REMOTE_UPSTREAM} {WORK_BRANCH}

#### Now the new release is in upstream repo

### Some manual steps follow to improve the UX

#### New relase on GitHub

Copy the following changelog lines to the description in form on link below
The release title is the new version.

In [ ]:
print('\n')
print('\n'.join(change_log))
print('\n\nhttps://github.com/theforeman/hammer-cli/releases/new?tag=%s' % NEW_VERSION)

In [ ]:
from IPython.display import Markdown as md

md('### Create new hammer-cli release in Redmine \n' + \
    '<a href="https://projects.theforeman.org/projects/hammer-cli/versions/new" target="_blank">https://projects.theforeman.org/projects/hammer-cli/versions/new</a>\n\n' + \
    'Set name to hammer-cli-%s' % (NEXT_FUTURE_VERSION if MAJOR_RELEASE else NEW_VERSION))

In [ ]:
if not MAJOR_RELEASE:
    print('Set fixed in versions to %s in following issues:' % NEW_VERSION)
    for i in referenced_issues:
        print('- https://projects.theforeman.org/issues/%s' % i)

## Congratulations

Release is public now. Please, don't forget to do packaging (https://github.com/theforeman/foreman-packaging).